In [ ]:
# Generates mlpModel
import onnxruntime as rt
from sklearn.metrics import accuracy_score
from scipy.stats import wilcoxon
import pandas as pd
import numpy as np
from testing import *
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn

data = pd.read_csv('data/investigation_train_large_checked.csv')
y = data['checked']
X = data.drop(['checked', 'Ja', 'Nee'], axis=1)
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Preprocessing
numeric_features = list(range(X.shape[1]))  # assuming all features are numeric in make_classification
numeric_transformer = StandardScaler()

# Combine preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
    ]
)

# Pipelines for both models
mlp_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", MLPClassifier(hidden_layer_sizes=(16,8),  
                    activation='relu',            
                    solver='adam',                
                    max_iter=100,                
                    random_state=42,
                    verbose=True))
])

# Train both models on full training data
mlp_pipeline.fit(X_train, y_train)

# Let's convert the model to ONNX
onnx_model = convert_sklearn(
    mlp_pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# How to get metrics such as accuracy
sess = rt.InferenceSession(onnx_model.SerializeToString())
y_pred_onnx =  sess.run(None, {'X': X_test.values.astype(np.float32)})
y_pred_onnx_binary = pred_to_binary(y_pred_onnx[0]) # Do not forget
y_test = y_test.astype(int) # Do not forget
accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx_binary)

print('Accuracy of the ONNX model: ', accuracy_onnx_model)

# Let's save the model
onnx.save(onnx_model, "model/mlpmodel.onnx")

In [ ]:
# Gemerates Random Forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from sklearn.metrics import accuracy_score
from skl2onnx import convert_sklearn
from goodModel import change_labels_data_augmentation_binary, data_augmentation_age
from sklearn.model_selection import train_test_split
import testing

def modify_data(df):
    df = change_labels_data_augmentation_binary(df, 'persoon_geslacht_vrouw')
    df = data_augmentation_age(df)
    #df = goodModel.drop_personality_columns(df)
    return df

df = pd.read_csv('data/investigation_train_large_checked.csv')
data = modify_data(df)


In [8]:

# Let's specify the features and the target
y = data['checked']
X = data.drop(['checked', 'Ja', 'Nee'], axis=1)
X = X.astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Preprocessing
numeric_features = list(range(X.shape[1]))  # assuming all features are numeric in make_classification
numeric_transformer = StandardScaler()

# Combine preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
    ]
)

# Pipelines for both models
rf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42))
])

# Train both models on full training data
rf_pipeline.fit(X_train, y_train)

# Let's convert the model to ONNX
onnx_model = convert_sklearn(
    rf_pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# How to get metrics such as accuracy
sess = rt.InferenceSession(onnx_model.SerializeToString())
y_pred_onnx =  sess.run(None, {'X': X_test.values.astype(np.float32)})
y_pred_onnx_binary = pred_to_binary(y_pred_onnx[0]) # Do not forget
y_test = y_test.astype(int) # Do not forget
accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx_binary)

print('Accuracy of the ONNX model: ', accuracy_onnx_model)

# Let's save the model
onnx.save(onnx_model, "model/RFmodel.onnx")

NameError: name 'pred_to_binary' is not defined